In [20]:
import pandas as pd
import json

# Data preprocessing

In [21]:
train_dir = '/Users/shirleenyoung/Desktop/Groningen/language technology project/argument_aspect_detection_v1.0/in_topic/train.jsonl'
dev_dir = '/Users/shirleenyoung/Desktop/Groningen/language technology project/argument_aspect_detection_v1.0/in_topic/dev.jsonl'
test_dir = '/Users/shirleenyoung/Desktop/Groningen/language technology project/argument_aspect_detection_v1.0/in_topic/test.jsonl'
data_train = pd.read_json(train_dir, lines=True)  
data_dev = pd.read_json(dev_dir, lines=True)
data_test = pd.read_json(test_dir, lines=True)

# data_train = data_train(data_train.aspect_pos.map(set(['no_Aspect']).issubset))
ind_train = data_train.aspect_pos.apply(lambda x: 'no_Aspect' not in x)
ind_dev = data_dev.aspect_pos.apply(lambda x: 'no_Aspect'  not in x)
ind_test = data_test.aspect_pos.apply(lambda x: 'no_Aspect' not in x)


data_train = data_train[ind_train]
data_dev = data_dev[ind_dev]
data_test =data_test[ind_test]

print('dataset \t\t\t number of no_aspect\t\t\tnumber of all sentences')
print('-'*100)
print(f'train \t\t\t\t {len(data_train)} \t\t\t\t {len(ind_train)}')
print(f'dev \t\t\t\t {len(data_dev)} \t\t\t\t {len(ind_dev)}')
print(f'test \t\t\t\t {len(data_test)} \t\t\t\t {len(ind_test)}')

dataset 			 number of no_aspect			number of all sentences
----------------------------------------------------------------------------------------------------
train 				 3466 				 3532
dev 				 492 				 500
test 				 991 				 1000


In [22]:
def get_data(keyword, df=data_train):
    data = df[[keyword]].values.tolist()
    data = [sample for samples in data for sample in samples]
    return data

X_train = get_data('sentence')
y_train = get_data('aspect_pos_string')
X_train_topic = get_data('topic')
X_train_stance = get_data('stance')
X_train_asp_pos = get_data('aspect_pos')

X_dev = get_data('sentence', data_dev)
y_dev = get_data('aspect_pos_string', data_dev)
X_dev_topic = get_data('topic', data_dev)
X_dev_stance = get_data('stance', data_dev)
X_dev_asp_pos = get_data('aspect_pos', data_dev)

X_test = get_data('sentence', data_test)
y_test = get_data('aspect_pos_string', data_test)
X_test_topic = get_data('topic', data_test)
X_test_stance = get_data('stance', data_test)
X_test_asp_pos = get_data('aspect_pos', data_test)

# NER TASK

In [23]:
import re
def get_asp_pos(split_asp_pos):
    pos_list =[]
    for sent in split_asp_pos:
        sent_list =[]
        sent_list.append(0)
        for word in sent:
            nums = re.findall(r'\d+', word)
            for i, num in enumerate(nums):
                if i%2==0:
                    sent_list.append(int(num))
                else:
                    sent_list.append(int(nums[i-1])+int(num))
        pos_list.append(sent_list) 
    return pos_list

train_asp_pos = get_asp_pos(X_train_asp_pos)
dev_asp_pos = get_asp_pos(X_dev_asp_pos)
test_asp_pos = get_asp_pos(X_test_asp_pos)

In [24]:
train_asp_pos

[[0, 57, 80],
 [0, 40, 69, 107, 124],
 [0, 52, 74],
 [0, 24, 38, 103, 124],
 [0, 6, 10, 21, 38, 46, 69, 95, 106],
 [0, 91, 106, 123, 140],
 [0, 33, 49],
 [0, 50, 60, 64, 68, 81, 102, 119, 137],
 [0, 59, 78],
 [0, 0, 17],
 [0, 38, 43, 48, 57, 112, 137],
 [0, 4, 18, 51, 55, 56, 74, 125, 139],
 [0, 49, 75, 110, 126],
 [0, 49, 79, 106, 139],
 [0, 57, 69, 86, 94],
 [0, 62, 78, 129, 145, 160, 175],
 [0, 0, 6],
 [0, 0, 20, 61, 77],
 [0, 46, 66],
 [0, 57, 65, 66, 81, 102, 129],
 [0, 78, 96],
 [0, 15, 22, 23, 27, 35, 42],
 [0, 20, 32, 40, 59, 93, 112],
 [0, 216, 222],
 [0, 69, 74, 117, 134, 183, 188, 207, 234],
 [0, 27, 43, 125, 129, 135, 144, 154, 167],
 [0, 39, 54],
 [0, 106, 119, 133, 141, 148, 161, 201, 214],
 [0, 22, 39, 49, 76],
 [0, 76, 93, 188, 204],
 [0, 103, 132],
 [0, 180, 187, 188, 202],
 [0, 16, 40],
 [0, 52, 67],
 [0, 33, 41, 46, 68],
 [0, 4, 26, 139, 151],
 [0, 30, 55, 56, 62],
 [0, 182, 192, 193, 214],
 [0, 0, 15, 40, 57, 120, 143],
 [0, 43, 55],
 [0, 195, 202, 203, 216],
 [0, 8

In [27]:
def get_tag(X_split,split_asp_pos):
    data = get_data('sentence', X_split)
    tag_data=[]
    for i, sent in enumerate(data):
        sent_dic ={}
        for j in range(len(split_asp_pos[i])-1):
            if j%2!=0:
                sent_dic[sent[split_asp_pos[i][j]:split_asp_pos[i][j+1]]] = 'I-ASP' 
            else:
                sent_dic[sent[split_asp_pos[i][j]:split_asp_pos[i][j+1]]] ='O'
        tag_data.append(sent_dic)
    
    single_tagged =[]
    for sample in tag_data:
        single=[]
        for key, value in sample.items():
            keys = key.rstrip().split(' ')
            for i, k in enumerate(keys):
                if i==0 and value =='I-ASP':
                    single.append((k, 'B-ASP'))
                else:
                    single.append((k, value))
        single_tagged.append(single)
    return single_tagged
    

In [28]:
train_tag = get_tag(data_train, train_asp_pos)
dev_tag = get_tag(data_dev, dev_asp_pos)
test_tag = get_tag(data_test, test_asp_pos)



In [30]:

def generate_file(file_name, ner_label, sent_split):
    with open (file_name, 'w') as f:
        for i, sent in enumerate(ner_label):
            for (key, value) in sent:
                f.write(sent_split[i]+' **'+key+' **'+value+'\n')


generate_file('tagged_ner_train.tsv', train_tag, X_train)
generate_file('tagged_ner_dev.tsv', dev_tag, X_dev)
generate_file('tagged_ner_test.tsv', test_tag, X_test)